# TF reporter activity analysis
### Aim
~36,000 reporters for 86 TFs were transfected into 9 different cell types & tested in 97 different perturbation conditions. In this script I will analyze TF reporter activities in detail and review how individual reporters respond to TF concentration variations.

### Data preprocessing

---
Load libraries

In [22]:
# Load libraries:
import matplotlib.pyplot as plt  # Equivalent to RColorBrewer and ggplot2
import pandas as pd  # Equivalent to dplyr, tibble, and readr
import seaborn as sns  # Equivalent to pheatmap and ggpubr
import plotly  # Equivalent to plotly
import numpy as np  # Equivalent to maditr
import string  # Equivalent to stringr
import re  # Equivalent to stringr
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

---
**Load data frames**

In [5]:
# Import processed bc counts from the preprocessing step
cDNA_df = pd.read_csv("/DATA/usr/m.trauernicht/projects/SuRE-TF/data/gcf7124_stimulations/results/mt20240124_reporter_activity_filt_combined.csv")

# We are not going to use NIH3T3 data, so remove it for now
cDNA_df = cDNA_df[cDNA_df['cell'] != "NIH3T3"]

# Rename stimulation status of control conditions
cDNA_df['stimulation'].fillna("no", inplace=True)

# Load RNA-seq data
tf_rna = pd.read_csv("/DATA/usr/m.trauernicht/data/RNA_seq/rna_tpm_all_tfs.tsv", sep='\t')

# Prepare data frame for following analyses
cDNA_df2 = cDNA_df.copy()
cDNA_df2['tf'] = cDNA_df2['tf'].str.replace('_.*', '', regex=True)
cDNA_df2 = cDNA_df2.loc[
    (cDNA_df2['neg_ctrls'] == "No") &
    (cDNA_df2['hPGK'] == "No") &
    (~cDNA_df2['tf'].str.contains('RANDOM', case=False, na=False)) &
    (cDNA_df2['native_enhancer'] == "No")]
cDNA_df2 = cDNA_df2[['tf', 'condition', 'stimulation', 'reference_condition', 'tf_target', 'effect_size', 'off_target', 'cell', 'reporter_id', 'commercial_reporter', 'reporter_activity_minP', 'gcf', 'reporter_dif_minP']]
cDNA_df2['reporter_activity_minP'] = cDNA_df2['reporter_activity_minP'].apply(lambda x: np.log2(x))
cDNA_df2 = cDNA_df2.drop_duplicates()

#cDNA_df2

---
**Load reference activities**

In [6]:
## Define optimal candidate conditions for each TF: either the highest expressing cell line, or the stimulated condition
### Select most active stimulation condition per TF
ref_conditions1 = cDNA_df2[cDNA_df2['tf_target'] == 1]
ref_conditions1 = ref_conditions1[['tf', 'condition', 'cell', 'reporter_id', 'commercial_reporter', 'reporter_activity_minP']]
ref_conditions1 = ref_conditions1.drop_duplicates()
ref_conditions1 = ref_conditions1.groupby(['tf', 'condition']).apply(lambda x: x.sort_values(by='reporter_activity_minP', ascending=False))
ref_conditions1.reset_index(drop=True, inplace=True)
ref_conditions1 = ref_conditions1.groupby(['tf', 'condition']).head(10)
ref_conditions1['median_reporter_activity_minP'] = ref_conditions1.groupby(['tf', 'condition'])['reporter_activity_minP'].transform('median')
ref_conditions1.reset_index(drop=True, inplace=True)
ref_conditions1 = ref_conditions1.groupby(['tf']).head(1)
ref_conditions1 = ref_conditions1[['tf', 'condition']]
ref_conditions1 = ref_conditions1.drop_duplicates()

### Top 3 conditions per TF based on highest TPM and highest median reporter activity
ref_conditions2 = pd.merge(cDNA_df2[cDNA_df2['stimulation'] == "no"], tf_rna.rename(columns={'tf': 'tf'}), on=['cell', 'tf'])
ref_conditions2['mean_reporter_activity_minP'] = ref_conditions2.groupby(['tf', 'condition'])['reporter_activity_minP'].transform('median')
ref_conditions2 = ref_conditions2[['mean_reporter_activity_minP', 'tf', 'condition', 'nTPM']].drop_duplicates()
ref_conditions2 = ref_conditions2.groupby(['tf']).apply(lambda x: x.sort_values('nTPM', ascending=False))
ref_conditions2.reset_index(drop=True, inplace=True)
ref_conditions2 = ref_conditions2.groupby(['tf']).head(3)
ref_conditions2 = ref_conditions2.groupby('tf').apply(lambda x: x.sort_values('mean_reporter_activity_minP', ascending=False))
ref_conditions2.reset_index(drop=True, inplace=True)
ref_conditions2 = ref_conditions2.groupby(['tf']).head(2)

ref_conditions2 = ref_conditions2[['tf', 'condition']].drop_duplicates()

ref_conditions = pd.concat([ref_conditions1, ref_conditions2])
ref_conditions = ref_conditions[['tf', 'condition']].drop_duplicates()

new_data = pd.DataFrame({
    'tf': ["PAX6", "ESRRB", "SMAD4", "GATA1", "RUNX2", "NR5A2", "SMAD2::SMAD3::SMAD4", "RFX1",
           "FOXO1", "FOSL1", "ETS2", "ONECUT1"],
    'condition': ["HepG2_NT", "mES_serum_2i_LIF", "HepG2_NT", "K562_Hemin", "U2OS", "mES_2i_LIF", "HepG2_NT", "mES_2i_LIF",
                  "mES_FOXA1-OE", "mES_2i_LIF", "HepG2_NT", "HepG2_NT"]
})

ref_conditions = pd.concat([ref_conditions, new_data])

ref_conditions = ref_conditions.sort_values('tf')

#ref_conditions

---
**Load perturbation fold-changes**

In [7]:
# Data frame with off-target activities
off_target_activities = cDNA_df2[cDNA_df2['tf_target'] == 2]
off_target_activities = off_target_activities[~((off_target_activities['tf'] == "SOX2") & (off_target_activities['condition'] == "mES_POU2F1"))]
off_target_activities['effect_size'] = pd.to_numeric(off_target_activities['effect_size'], errors='coerce')
off_target_activities['effect_size'] = off_target_activities['effect_size'].fillna(0)
off_target_activities['reporter_dif_mean'] = off_target_activities.groupby(['tf', 'condition'])['reporter_dif_minP'].transform(lambda x: np.median(x.dropna()))
off_target_activities['reporter_dif_mean'] = np.where(off_target_activities['effect_size'] == 0, -off_target_activities['reporter_dif_mean'], off_target_activities['reporter_dif_mean'])
off_target_activities = off_target_activities.groupby('tf').apply(lambda x: x.sort_values('reporter_dif_mean', ascending=False).head(1))
off_target_activities = off_target_activities[['reporter_id', 'tf', 'commercial_reporter', 'reporter_dif_minP', 'condition', 'effect_size']]
off_target_activities = off_target_activities.drop_duplicates()

#off_target_activities

on_target_activities = cDNA_df2[cDNA_df2['tf_target'] == 1]
on_target_activities['effect_size'] = pd.to_numeric(on_target_activities['effect_size'], errors='coerce')
on_target_activities['reporter_dif_mean'] = on_target_activities.groupby(['tf', 'condition'])['reporter_dif_minP'].transform(lambda x: np.median(x.dropna()))
on_target_activities['reporter_dif_mean'] = np.where(on_target_activities['effect_size'] == 0, -on_target_activities['reporter_dif_mean'], on_target_activities['reporter_dif_mean'])
on_target_activities = on_target_activities.groupby('tf').apply(lambda x: x.sort_values('reporter_dif_mean', ascending=False).head(1))
on_target_activities = on_target_activities[['reporter_id', 'tf', 'commercial_reporter', 'reporter_dif_minP', 'condition', 'effect_size']]
on_target_activities = on_target_activities.drop_duplicates()

#on_target_activities

---
**Load RNA-seq data**

In [24]:
# Prepare tf reporter data frame
cDNA_df2 = cDNA_df.copy()
cDNA_df2 = cDNA_df2[(cDNA_df2['neg_ctrls'] == "No") &
                    (cDNA_df2['stimulation'] == "no") &
                    (cDNA_df2['hPGK'] == "No") &
                    (~cDNA_df2['tf'].str.contains('RANDOM', case=False, na=False)) &
                    (cDNA_df2['native_enhancer'] == "No")]
cDNA_df2 = cDNA_df2[['tf', 'reporter_activity_minP', 'cell', 'reporter_id', 'commercial_reporter', 'spacing', 'promoter', 'background', 'distance']]
cDNA_df2['reporter_activity_minP'] = cDNA_df2.groupby(['cell', 'reporter_id'])['reporter_activity_minP'].transform('mean')
cDNA_df2['reporter_activity_minP'] = np.log2(cDNA_df2['reporter_activity_minP'])
cDNA_df2 = cDNA_df2.drop_duplicates()
cDNA_df2['tf'] = cDNA_df2['tf'].str.replace('_.*', '', regex=True)


# Combine TF reporter data with RNA-seq data
cDNA_df3 = pd.merge(cDNA_df2, tf_rna, how='left')
cDNA_df3['TPM'] = cDNA_df3['TPM'].fillna(0)
cDNA_df3['nTPM'] = cDNA_df3['nTPM'].fillna(0)

# Compute pearson correlations for each individual reporter of reporter activity vs. nTPM counts across the nine cell types
for i in cDNA_df3['reporter_id'].unique():
    if cDNA_df3[cDNA_df3['reporter_id'] == i].shape[0] > 4:
        cor, p_value = pearsonr(2**cDNA_df3[cDNA_df3['reporter_id'] == i]['reporter_activity_minP'],
                                cDNA_df3[cDNA_df3['reporter_id'] == i]['nTPM'])
        cDNA_df3.loc[cDNA_df3['reporter_id'] == i, 'cor_pval'] = p_value
        cDNA_df3.loc[cDNA_df3['reporter_id'] == i, 'cor'] = cor
    else:
        cDNA_df3.loc[cDNA_df3['reporter_id'] == i, 'cor_pval'] = 1
        cDNA_df3.loc[cDNA_df3['reporter_id'] == i, 'cor'] = 0



# Set all negative correlations to zero
cDNA_df3['cor_pval'] = np.where(cDNA_df3['cor_pval'] == cDNA_df3['cor'], 1, cDNA_df3['cor_pval'])
cDNA_df3['cor_pval_fdr'] = cDNA_df3.groupby('tf')['cor_pval'].transform(lambda x: multipletests(x, method='fdr_bh')[1])
cDNA_df3['cor_pval'] = np.where(cDNA_df3['cor'] < 0, 1, cDNA_df3['cor_pval'])
cDNA_df3['cor_pval_fdr'] = np.where(cDNA_df3['cor'] < 0, 1, cDNA_df3['cor_pval_fdr'])
cDNA_df3['cor_pval'] = -np.log10(cDNA_df3['cor_pval'])
cDNA_df3['cor_pval_fdr'] = -np.log10(cDNA_df3['cor_pval_fdr'])
cDNA_df3['cor_pval'] = np.where(cDNA_df3['cor_pval'].isna(), 1, cDNA_df3['cor_pval'])
cDNA_df3['cor_pval_fdr'] = np.where(cDNA_df3['cor_pval_fdr'].isna(), 1, cDNA_df3['cor_pval_fdr'])
cDNA_df3['cor'] = np.where(cDNA_df3['cor'].isna(), 1, cDNA_df3['cor'])

# Filter for TFs where correlations actually make sense
rna_correlations_df = cDNA_df3.copy()
rna_correlations_df['min_TPM'] = rna_correlations_df.groupby('tf')['nTPM'].transform('min')
rna_correlations_df['max_TPM'] = rna_correlations_df.groupby('tf')['nTPM'].transform('max')
rna_correlations_df['dif_TPM'] = rna_correlations_df['max_TPM'] - rna_correlations_df['min_TPM']
rna_correlations_df['mean_tf_activity'] = rna_correlations_df.groupby(['tf', 'cell'])['reporter_activity_minP'].transform('mean')
rna_correlations_df['max_mean_tf_activity'] = rna_correlations_df.groupby('tf')['mean_tf_activity'].transform('max')
rna_correlations_df['reasonable'] = np.where((rna_correlations_df['max_TPM'] > 8) &
                                             (rna_correlations_df['dif_TPM'] > 5) &
                                             (rna_correlations_df['min_TPM'] < 1) &
                                             (rna_correlations_df['max_mean_tf_activity'] > 0.75) |
                                             (rna_correlations_df['tf'].isin(["GATA1", "NFATC1", "GLI1", "STAT3", "SP1", "FOXA1", "TEAD1", "NFKB1", "ZFX", "NR4A1"])),
                                             "Yes", "No")
rna_correlations_df['reasonable'] = np.where(rna_correlations_df['tf'].isin(["TCF7L2", "TCF7", "TFEB", "TP53", "GATA3", "CEBPA"]),
                                             "No", rna_correlations_df['reasonable'])

reasonable_corr = rna_correlations_df[rna_correlations_df['reasonable'] == "No"]

# Export correlations
rna_correlations = rna_correlations_df[rna_correlations_df['reasonable'] == "Yes"]
rna_correlations = rna_correlations[['reporter_id', 'cor_pval', 'cor']]
rna_correlations['cor_pval'] = np.where(rna_correlations['reporter_id'].isin(reasonable_corr['reporter_id']), 0, rna_correlations['cor_pval'])
rna_correlations['cor'] = np.where(rna_correlations['reporter_id'].isin(reasonable_corr['reporter_id']), 0, rna_correlations['cor'])

# Write to file
rna_correlations.to_csv("/DATA/usr/m.trauernicht/projects/SuRE-TF/data/rna_correlations.tsv", sep='\t', index=False)
rna_correlations_df.to_csv("/DATA/usr/m.trauernicht/projects/SuRE-TF/data/rna_correlations_df.tsv", sep='\t', index=False)

In [29]:
rna_correlations_df.to_csv("/DATA/usr/m.trauernicht/projects/SuRE-TF/data/rna_correlations_df.tsv", sep='\t', index=False)